Task 2

In [29]:
import numpy as np
import heapq

# Define campus locations (nodes)
nodes = {
    0: "Main Entrance",
    1: "Morgan's Walk",
    2: "Building C",
    3: "Building G",
    4: "Building U",
    5: "Library",
    6: "Student Central",
    7: "Wheelchair Accessible Toilet Student Central",
    8: "Wheelchair Accessible Sage Cafe",
    9: "Building LC",
    10: "Wheelchair Accessible Fusion Cafe",
    11: "Building LA",
    12: "Building LB",
    13: "Building EC",
    14: "Building EA",
    15: "Car Park MA",
    16: "Building MB",
    17: "Deakin Active Gym",
    18: "Building MC",
    19: "Residential Basketball Court",
    20: "Deakin Residential Car Park"
}

# Define (x,y) coordinates for each location for heuristic calculation
# Using a grid representation (meters from an arbitrary origin)
coordinates = {
    0: (10, 20),     # Main Entrance
    1: (30, 40),     # Morgan's Walk
    2: (60, 50),     # Building C
    3: (90, 60),     # Building G
    4: (120, 70),    # Building U
    5: (150, 50),    # Library
    6: (100, 90),    # Student Central
    7: (80, 110),    # Wheelchair Accessible Toilet
    8: (60, 130),    # Wheelchair Accessible Sage Cafe
    9: (110, 120),   # Building LC
    10: (140, 110),  # Wheelchair Accessible Fusion Cafe
    11: (170, 100),  # Building LA
    12: (200, 90),   # Building LB
    13: (180, 130),  # Building EC
    14: (150, 150),  # Building EA
    15: (120, 160),  # Car Park MA
    16: (90, 170),   # Building MB
    17: (60, 180),   # Deakin Active Gym
    18: (30, 150),   # Building MC
    19: (20, 120),   # Residential Basketball Court
    20: (50, 100),   # Deakin Residential Car Park
}

# Create an adjacency matrix to represent the wheelchair-accessible paths
num_locations = len(nodes)
adjacency_matrix = np.full((num_locations, num_locations), float('inf'))

# Define path connections and their distances (meters)
# The distances account for accessibility factors
# Format: (location1, location2, distance)
connections = [
    (0, 1, 10),    # Main Entrance to Morgan's Walk
    (1, 2, 15),    # Morgan's Walk to Building C
    (2, 3, 15),    # Building C to Building G
    (3, 4, 45),    # Building G to Building U
    (4, 5, 55),    # Building U to Library
    (0, 5, 35),    # Main Entrance to Library
    (0, 4, 20),    # Main Entrance to Building U
    (1, 6, 35),    # Library to Student Central
    (6, 7, 5),     # Student Central to Wheelchair Accessible Toilet Student Central
    (7, 8, 10),    # Wheelchair Accessible Toilet Student Central to Wheelchair Accessible Sage Cafe
    (8, 9, 55),    # Wheelchair Accessible Sage Cafe to Building LC
    (9, 10, 20),   # Building LC to Wheelchair Accessible Fusion Cafe
    (10, 11, 20),  # Wheelchair Accessible Fusion Cafe to Building LA
    (11, 12, 20),  # Building LA to Building LB
    (5, 11, 70),   # Library to Building LA
    (11, 13, 30),  # Building LA to Building EC
    (13, 14, 20),  # Building EC to Building EA
    (14, 15, 32),  # Building EA to Car Park MA
    (15, 16, 27),  # Car Park MA to Building MB
    (16, 17, 14),  # Building MB to Deakin Active Gym
    (16, 18, 20),  # Building MB to Building MC
    (17, 18, 34),  # Deakin Active Gym to Building MC
    (18, 19, 27),  # Building MC to Residential Basketball Court
    (19, 20, 8),   # Residential Basketball Court to Deakin Residential Car Park
    (20, 7, 98),   # Deakin Residential Car Park to Wheelchair Accessible Toilet Student Central
    (6, 9, 40),    # Student Central to Building LC
    (9, 13, 35),   # Building LC to Building EC
    (3, 6, 50),    # Building G to Student Central
    (1, 20, 108),  # Morgan's Walk to Deakin Residential Car Park
    (2, 8, 64),    # Building C to Wheelchair Accessible Sage Cafe
    (4, 10, 90),   # Building U to Wheelchair Accessible Fusion Cafe
    (15, 18, 38),  # Car Park MA to Building MC
    (12, 14, 35),  # Building LB to Building EA 
]

# Fill the adjacency matrix with the connection distances
for start, end, distance in connections:
    adjacency_matrix[start, end] = distance
    adjacency_matrix[end, start] = distance  # Assuming bidirectional paths

# A* Algorithm Implementation
def heuristic(node, goal):
    """Calculate the Euclidean distance between two nodes"""
    x1, y1 = coordinates[node]
    x2, y2 = coordinates[goal]
    return np.sqrt((x2 - x1)**2 + (y2 - y1)**2)

def a_star(start, goal):
    """A* algorithm implementation for finding the shortest path"""
    if start not in range(num_locations) or goal not in range(num_locations):
        return None, float('inf')
    
    # Initialize open and closed sets
    open_set = []
    closed_set = set()
    
    # Calculate g and f values for the start node
    g_scores = {start: 0}
    f_scores = {start: heuristic(start, goal)}
    
    # For reconstructing the path
    came_from = {}
    
    # Add start node to open set
    heapq.heappush(open_set, (f_scores[start], start))
    
    while open_set:
        # Get the node with the lowest f score from the open set
        current_f, current = heapq.heappop(open_set)
        
        # If we've reached the goal, reconstruct the path
        if current == goal:
            path = []
            while current in came_from:
                path.append(current)
                current = came_from[current]
            path.append(start)
            path.reverse()
            return path, g_scores[goal]
        
        # Add current node to closed set
        closed_set.add(current)
        
        # Check all neighbors
        for neighbor in range(num_locations):
            # Skip if there's no direct connection
            if adjacency_matrix[current, neighbor] == float('inf'):
                continue
            
            # Skip if neighbor is already evaluated
            if neighbor in closed_set:
                continue
            
            # Calculate tentative g score
            tentative_g = g_scores[current] + adjacency_matrix[current, neighbor]
            
            # If neighbor is not in open set or has a better g score
            if neighbor not in g_scores or tentative_g < g_scores[neighbor]:
                # Record the path
                came_from[neighbor] = current
                g_scores[neighbor] = tentative_g
                f_scores[neighbor] = g_scores[neighbor] + heuristic(neighbor, goal)
                
                # Add to open set if not already there
                if not any(node == neighbor for _, node in open_set):
                    heapq.heappush(open_set, (f_scores[neighbor], neighbor))
    
    # If we get here, there's no path
    return None, float('inf')

# Test the A* implementation with different start and end points
def test_navigation():
    print("\nWheelchair-Accessible Campus Navigation")
    print("----------------------------------------")

    # Print available locations
    print("\nAvailable Locations:")
    for node_id, name in nodes.items():
        print(f"{node_id}: {name}")
    
    while True:
        try:
            # User input for start and goal
            start_input = input("\nEnter the number for START location (or type 'exit' to quit): ").strip()
            if start_input.lower() == 'exit':
                print("Thank you for using the campus navigation system")
                break
            start = int(start_input)

            goal_input = input("Enter the number for DESTINATION location: ").strip()
            goal = int(goal_input)

            if start not in nodes or goal not in nodes:
                print("Invalid input. Please enter valid location numbers.")
                continue

            # Run A* search
            path, distance = a_star(start, goal)
            
            print(f"\nPath from {nodes[start]} to {nodes[goal]}:")
            if path:
                path_names = [nodes[node] for node in path]
                print(" -> ".join(path_names))
                print(f"Cost: Total distance = {distance:.2f} meters")
            else:
                print("No path found between the selected locations.")

        except ValueError:
            print("Invalid input. Please enter numeric IDs.")

# Run the tests
test_navigation()


Wheelchair-Accessible Campus Navigation
----------------------------------------

Available Locations:
0: Main Entrance
1: Morgan's Walk
2: Building C
3: Building G
4: Building U
5: Library
6: Student Central
7: Wheelchair Accessible Toilet Student Central
8: Wheelchair Accessible Sage Cafe
9: Building LC
10: Wheelchair Accessible Fusion Cafe
11: Building LA
12: Building LB
13: Building EC
14: Building EA
15: Car Park MA
16: Building MB
17: Deakin Active Gym
18: Building MC
19: Residential Basketball Court
20: Deakin Residential Car Park



Enter the number for START location (or type 'exit' to quit):  0
Enter the number for DESTINATION location:  3



Path from Main Entrance to Building G:
Main Entrance -> Building U -> Building G
Cost: Total distance = 65.00 meters



Enter the number for START location (or type 'exit' to quit):  1
Enter the number for DESTINATION location:  3



Path from Morgan's Walk to Building G:
Morgan's Walk -> Building C -> Building G
Cost: Total distance = 30.00 meters



Enter the number for START location (or type 'exit' to quit):  exit


Thank you for using the campus navigation system


Task 3

In [32]:
import numpy as np
import heapq

# Define campus locations (nodes)
nodes = {
    0: "Main Entrance",
    1: "Morgan's Walk",
    2: "Building C",
    3: "Building G",
    4: "Building U",
    5: "Library",
    6: "Student Central",
    7: "Wheelchair Accessible Toilet Student Central",
    8: "Wheelchair Accessible Sage Cafe",
    9: "Building LC",
    10: "Wheelchair Accessible Fusion Cafe",
    11: "Building LA",
    12: "Building LB",
    13: "Building EC",
    14: "Building EA",
    15: "Car Park MA",
    16: "Building MB",
    17: "Deakin Active Gym",
    18: "Building MC",
    19: "Residential Basketball Court",
    20: "Deakin Residential Car Park",
    21: "Bus Stop to Box Hill",
    22: "Entrance to the creek",
    23: "Building T",
    24: "Building BA",
    25: "Building HI",
    26: "Wheelchair Accessible toilet Pavillion Building",
    27: "Building B",
    28: "Building A",
    29: "Building P",
    30: "Building HE"
}

# Define (x,y) coordinates for each location
coordinates = {
    0: (10, 20),     # Main Entrance
    1: (30, 40),     # Morgan's Walk
    2: (60, 50),     # Building C
    3: (90, 60),     # Building G
    4: (120, 70),    # Building U
    5: (150, 50),    # Library
    6: (100, 90),    # Student Central
    7: (80, 110),    # Wheelchair Accessible Toilet Student Central
    8: (60, 130),    # Wheelchair Accessible Sage Cafe
    9: (110, 120),   # Building LC
    10: (140, 110),  # Wheelchair Accessible Fusion Cafe
    11: (170, 100),  # Building LA
    12: (200, 90),   # Building LB
    13: (180, 130),  # Building EC
    14: (150, 150),  # Building EA
    15: (120, 160),  # Car Park MA
    16: (90, 170),   # Building MB
    17: (60, 180),   # Deakin Active Gym
    18: (30, 150),   # Building MC
    19: (20, 120),   # Residential Basketball Court
    20: (50, 100),   # Deakin Residential Car Park
    21: (135, 175),  # Bus Stop to Box Hill
    22: (70, 90),    # Entrance to the creek
    23: (165, 60),   # Building T
    24: (160, 30),   # Building BA
    25: (180, 35),   # Building HI
    26: (130, 95),   # Wheelchair Accessible toilet Pavillion Building
    27: (40, 55),    # Building B
    28: (45, 70),    # Building A
    29: (130, 40),   # Building P
    30: (105, 100)   # Building HE
}

# Create an adjacency matrix to represent the wheelchair-accessible paths
num_locations = len(nodes)
adjacency_matrix = np.full((num_locations, num_locations), float('inf'))

# Define path connections and their distances (meters)
connections = [
    (0, 1, 10),    # Main Entrance to Morgan's Walk
    (1, 2, 15),    # Morgan's Walk to Building C
    (2, 3, 15),    # Building C to Building G
    (3, 4, 45),    # Building G to Building U
    (4, 5, 55),    # Building U to Library
    (0, 5, 35),    # Main Entrance to Library
    (0, 4, 20),    # Main Entrance to Building U
    (1, 6, 35),    # Morgan's Walk to Student Central
    (6, 7, 5),     # Student Central to Wheelchair Accessible Toilet Student Central
    (7, 8, 10),    # Wheelchair Accessible Toilet Student Central to Wheelchair Accessible Sage Cafe
    (8, 9, 55),    # Wheelchair Accessible Sage Cafe to Building LC
    (9, 10, 20),   # Building LC to Wheelchair Accessible Fusion Cafe
    (10, 11, 20),  # Wheelchair Accessible Fusion Cafe to Building LA
    (11, 12, 20),  # Building LA to Building LB
    (5, 11, 70),   # Library to Building LA
    (11, 13, 30),  # Building LA to Building EC
    (13, 14, 20),  # Building EC to Building EA
    (14, 15, 32),  # Building EA to Car Park MA
    (15, 16, 27),  # Car Park MA to Building MB
    (16, 17, 14),  # Building MB to Deakin Active Gym
    (16, 18, 20),  # Building MB to Building MC
    (17, 18, 34),  # Deakin Active Gym to Building MC
    (18, 19, 27),  # Building MC to Residential Basketball Court
    (19, 20, 8),   # Residential Basketball Court to Deakin Residential Car Park
    (20, 7, 98),   # Deakin Residential Car Park to Wheelchair Accessible Toilet Student Central
    (6, 9, 40),    # Student Central to Building LC
    (9, 13, 35),   # Building LC to Building EC
    (3, 6, 50),    # Building G to Student Central
    (1, 20, 108),  # Morgan's Walk to Deakin Residential Car Park
    (2, 8, 64),    # Building C to Wheelchair Accessible Sage Cafe
    (4, 10, 90),   # Building U to Wheelchair Accessible Fusion Cafe
    (15, 18, 38),  # Car Park MA to Building MC
    (12, 14, 35),  # Building LB to Building EA
    (15, 21, 15),  # Car Park MA to Bus Stop to Box Hill
    (20, 22, 15),  # Deakin Residential Car Park to Entrance to the Creek
    (4, 26, 25),   # Building U to Wheelchair Accessible toilet Pavillion Ground
    (1, 27, 30),   # Morgan's Walk to Building B
    (1, 28, 35),   # Morgan's Walk to Building A
    (27, 28, 5),   # Building B to Building A
    (5, 23, 15),   # Library to Building T
    (5, 24, 20),   # Library to Building BA
    (5, 25, 30),   # Library to Building HI
    (23, 24, 10),  # Building T to Building BA
    (24, 25, 10),  # Building BA to Building HI
    (5, 29, 15),   # Library to Building P
    (29, 6, 15),   # Building P to Student Central
    (6, 30, 5),    # Student Central to Building HE
    (25, 6, 60),   # Building HI to Student Central
]

# Define terrain features for path segments (edges)
# Types: 'flat', 'slope', 'obstacles', 'kerb_ramp', 'crossing'
terrain_features = {
    (0, 1): {'type': 'flat', 'effort_multiplier': 1.0},
    (1, 2): {'type': 'mild_slope', 'effort_multiplier': 1.3},
    (2, 3): {'type': 'flat', 'effort_multiplier': 1.0},
    (3, 4): {'type': 'flat', 'effort_multiplier': 1.0},
    (4, 5): {'type': 'flat', 'effort_multiplier': 1.0},
    (1, 6): {'type': 'mild_slope', 'effort_multiplier': 1.3},
    (6, 7): {'type': 'flat', 'effort_multiplier': 1.0},
    (7, 8): {'type': 'flat', 'effort_multiplier': 1.0},
    (8, 9): {'type': 'gravel', 'effort_multiplier': 1.6},
    (9, 10): {'type': 'flat', 'effort_multiplier': 1.0},
    (10, 11): {'type': 'flat', 'effort_multiplier': 1.0},
    (11, 12): {'type': 'flat', 'effort_multiplier': 1.0},
    (5, 11): {'type': 'flat', 'effort_multiplier': 1.0},
    (11, 13): {'type': 'kerb_ramp', 'effort_multiplier': 1.4},
    (13, 14): {'type': 'steep_slope', 'effort_multiplier': 1.8},
    (14, 15): {'type': 'flat', 'effort_multiplier': 1.0},
    (15, 16): {'type': 'flat', 'effort_multiplier': 1.0},
    (16, 17): {'type': 'kerb_ramp', 'effort_multiplier': 1.4},
    (17, 18): {'type': 'mild_slope', 'effort_multiplier': 1.3},
    (18, 19): {'type': 'crossing', 'effort_multiplier': 1.5},
    (19, 20): {'type': 'flat', 'effort_multiplier': 1.0},
    (20, 7): {'type': 'mild_slope', 'effort_multiplier': 1.3},
    (6, 9): {'type': 'flat', 'effort_multiplier': 1.0},
    (9, 13): {'type': 'steep_slope', 'effort_multiplier': 1.8},
    (3, 6): {'type': 'mild_slope', 'effort_multiplier': 1.3},
    (1, 20): {'type': 'gravel', 'effort_multiplier': 1.6},
    (2, 8): {'type': 'steep_slope', 'effort_multiplier': 1.8},
    (4, 10): {'type': 'kerb_ramp', 'effort_multiplier': 1.4},
    (15, 18): {'type': 'gravel', 'effort_multiplier': 1.6},
    (12, 14): {'type': 'mild_slope', 'effort_multiplier': 1.3},
    (0, 19): {'type': 'mild_slope', 'effort_multiplier': 1.3},
    (5, 10): {'type': 'flat', 'effort_multiplier': 1.0},
    (2, 20): {'type': 'crossing', 'effort_multiplier': 1.5},
    (6, 14): {'type': 'steep_slope', 'effort_multiplier': 1.8},
    (8, 17): {'type': 'gravel', 'effort_multiplier': 1.6},
    (10, 13): {'type': 'kerb_ramp', 'effort_multiplier': 1.4},
    (12, 13): {'type': 'flat', 'effort_multiplier': 1.0},
    (7, 19): {'type': 'flat', 'effort_multiplier': 1.0},
    (3, 9): {'type': 'mild_slope', 'effort_multiplier': 1.3},
    (16, 19): {'type': 'steep_slope', 'effort_multiplier': 1.8},
    (4, 6): {'type': 'crossing', 'effort_multiplier': 1.5},
    (0, 2): {'type': 'mild_slope', 'effort_multiplier': 1.3},
}

# Fill the adjacency matrix with the connection distances
for start, end, distance in connections:
    adjacency_matrix[start, end] = distance
    adjacency_matrix[end, start] = distance  # Assuming bidirectional paths
    
    # Add matching terrain feature for reverse direction if not already defined
    if (end, start) not in terrain_features and (start, end) in terrain_features:
        terrain_features[(end, start)] = terrain_features[(start, end)]

# Original A* Heuristic: Euclidean distance
def original_heuristic(node, goal):
    """Calculate the Euclidean distance between two nodes"""
    x1, y1 = coordinates[node]
    x2, y2 = coordinates[goal]
    return np.sqrt((x2 - x1)**2 + (y2 - y1)**2)

# New A* Heuristic: Accounts for terrain features
def enhanced_heuristic(node, goal):
    """
    Enhanced heuristic considering terrain difficulty.
    It adds a weighted Euclidean distance by estimating effort due to terrain.
    """
    x1, y1 = coordinates[node]
    x2, y2 = coordinates[goal]
    base_distance = np.sqrt((x2 - x1)**2 + (y2 - y1)**2)
    
    # Estimate average terrain multiplier toward the goal
    multiplier = 1.0
    for neighbor in range(num_locations):
        if adjacency_matrix[node][neighbor] != float('inf'):
            if (node, neighbor) in terrain_features:
                multiplier = max(multiplier, terrain_features[(node, neighbor)]['effort_multiplier'])

    return base_distance * multiplier

# A* Algorithm Implementation with selectable heuristic
def a_star(start, goal, use_enhanced_heuristic=False, consider_terrain=False):
    """
    A* algorithm implementation for finding the optimal wheelchair-accessible path
    
    Parameters:
    - start: Starting node ID
    - goal: Goal node ID
    - use_enhanced_heuristic: Whether to use the enhanced terrain-aware heuristic
    - consider_terrain: Whether to account for terrain features in path cost
    
    Returns:
    - path: List of node IDs from start to goal
    - distance: Total physical distance of the path
    - effort: Total effort cost considering terrain (if enabled)
    """
    if start not in range(num_locations) or goal not in range(num_locations):
        return None, float('inf'), float('inf')
    
    # Choose appropriate heuristic function
    heuristic_fn = enhanced_heuristic if use_enhanced_heuristic else original_heuristic
    
    # Initialize data structures
    open_set = []
    closed_set = set()
    g_scores = {start: 0}
    f_scores = {start: heuristic_fn(start, goal)}
    came_from = {}
    effort_scores = {start: 0}
    
    # Add start node to open set
    heapq.heappush(open_set, (f_scores[start], start))
    
    while open_set:
        # Get node with lowest f score
        current_f, current = heapq.heappop(open_set)
        
        # Check if we've reached the goal
        if current == goal:
            # Reconstruct path
            path = []
            while current in came_from:
                path.append(current)
                current = came_from[current]
            path.append(start)
            path.reverse()
            
            return path, g_scores[goal], effort_scores[goal]
        
        # Add current node to closed set
        closed_set.add(current)
        
        # Check all neighbors
        for neighbor in range(num_locations):
            # Skip if no direct connection or already evaluated
            if adjacency_matrix[current, neighbor] == float('inf') or neighbor in closed_set:
                continue
            
            # Get basic distance
            distance = adjacency_matrix[current, neighbor]
            
            # Calculate terrain-adjusted effort if enabled
            if consider_terrain and (current, neighbor) in terrain_features:
                effort = distance * terrain_features[(current, neighbor)]['effort_multiplier']
            else:
                effort = distance
            
            # Calculate scores
            tentative_g = g_scores[current] + distance
            tentative_effort = effort_scores[current] + effort
            
            # Check if this path is better
            if neighbor not in g_scores or tentative_g < g_scores[neighbor]:
                # Update path
                came_from[neighbor] = current
                g_scores[neighbor] = tentative_g
                effort_scores[neighbor] = tentative_effort
                
                # Choose appropriate cost for f-score calculation
                path_cost = tentative_effort if consider_terrain else tentative_g
                f_scores[neighbor] = path_cost + heuristic_fn(neighbor, goal)
                
                # Add to open set if not already there
                if not any(node == neighbor for _, node in open_set):
                    heapq.heappush(open_set, (f_scores[neighbor], neighbor))
    
    # No path found
    return None, float('inf'), float('inf')


# Function to compare the two navigation approaches
def compare_navigation():
    print("\nWheelchair-Accessible Campus Navigation")
    print("----------------------------------------")

    # Print available locations
    print("\nAvailable Locations:")
    for node_id, name in nodes.items():
        print(f"{node_id}: {name}")
    
    while True:
        try:
            # User input for start and goal
            start_input = input("\nEnter the number for START location (or type 'exit' to quit): ").strip()
            if start_input.lower() == 'exit':
                print("Thank you for using the campus navigation system")
                break
            start = int(start_input)

            goal_input = input("Enter the number for DESTINATION location: ").strip()
            goal = int(goal_input)

            if start not in nodes or goal not in nodes:
                print("Invalid input. Please enter valid location numbers.")
                continue
                
            # Original A* (distance-only)
            path1, distance1, _ = a_star(start, goal, use_enhanced_heuristic=False, consider_terrain=False)
            
            # Enhanced A* (terrain-aware)
            path2, distance2, effort2 = a_star(start, goal, use_enhanced_heuristic=True, consider_terrain=True)

            print(f"\nPath from {nodes[start]} to {nodes[goal]}:")
            if path1 and path2:
                path1_names = [nodes[node] for node in path1]
                path2_names = [nodes[node] for node in path2]
                
                print("Original Path (Distance only):")
                print(" -> ".join(path1_names))
                print(f"Distance: {distance1} meters")
                
                print("\nEnhanced Path (Terrain-aware):")
                print(" -> ".join(path2_names))
                print(f"Distance: {distance2} meters (Effort: {effort2:.2f})")
            else:
                print("No path found between the selected locations.")

        except ValueError:
            print("Invalid input. Please enter numeric IDs.")

# Run the comparison
compare_navigation()


Wheelchair-Accessible Campus Navigation
----------------------------------------

Available Locations:
0: Main Entrance
1: Morgan's Walk
2: Building C
3: Building G
4: Building U
5: Library
6: Student Central
7: Wheelchair Accessible Toilet Student Central
8: Wheelchair Accessible Sage Cafe
9: Building LC
10: Wheelchair Accessible Fusion Cafe
11: Building LA
12: Building LB
13: Building EC
14: Building EA
15: Car Park MA
16: Building MB
17: Deakin Active Gym
18: Building MC
19: Residential Basketball Court
20: Deakin Residential Car Park
21: Bus Stop to Box Hill
22: Entrance to the creek
23: Building T
24: Building BA
25: Building HI
26: Wheelchair Accessible toilet Pavillion Building
27: Building B
28: Building A
29: Building P
30: Building HE



Enter the number for START location (or type 'exit' to quit):  10
Enter the number for DESTINATION location:  9



Path from Wheelchair Accessible Fusion Cafe to Building LC:
Original Path (Distance only):
Wheelchair Accessible Fusion Cafe -> Building LC
Distance: 20.0 meters

Enhanced Path (Terrain-aware):
Wheelchair Accessible Fusion Cafe -> Building LC
Distance: 20.0 meters (Effort: 20.00)



Enter the number for START location (or type 'exit' to quit):  exit


Thank you for using the campus navigation system


Task 4

In [35]:
import numpy as np
import heapq

# Define campus locations (nodes)
nodes = {
    0: "Main Entrance",
    1: "Morgan's Walk",
    2: "Building C",
    3: "Building G",
    4: "Building U",
    5: "Library",
    6: "Student Central",
    7: "Wheelchair Accessible Toilet Student Central",
    8: "Wheelchair Accessible Sage Cafe",
    9: "Building LC",
    10: "Wheelchair Accessible Fusion Cafe",
    11: "Building LA",
    12: "Building LB",
    13: "Building EC",
    14: "Building EA",
    15: "Car Park MA",
    16: "Building MB",
    17: "Deakin Active Gym",
    18: "Building MC",
    19: "Residential Basketball Court",
    20: "Deakin Residential Car Park",
    21: "Bus Stop to Box Hill",
    22: "Entrance to the creek",
    23: "Building T",
    24: "Building BA",
    25: "Building HI",
    26: "Wheelchair Accessible toilet Pavillion Building",
    27: "Building B",
    28: "Building A",
    29: "Building P",
    30: "Building HE"
}

# Define (x,y) coordinates for each location
coordinates = {
    0: (10, 20),     # Main Entrance
    1: (30, 40),     # Morgan's Walk
    2: (60, 50),     # Building C
    3: (90, 60),     # Building G
    4: (120, 70),    # Building U
    5: (150, 50),    # Library
    6: (100, 90),    # Student Central
    7: (80, 110),    # Wheelchair Accessible Toilet Student Central
    8: (60, 130),    # Wheelchair Accessible Sage Cafe
    9: (110, 120),   # Building LC
    10: (140, 110),  # Wheelchair Accessible Fusion Cafe
    11: (170, 100),  # Building LA
    12: (200, 90),   # Building LB
    13: (180, 130),  # Building EC
    14: (150, 150),  # Building EA
    15: (120, 160),  # Car Park MA
    16: (90, 170),   # Building MB
    17: (60, 180),   # Deakin Active Gym
    18: (30, 150),   # Building MC
    19: (20, 120),   # Residential Basketball Court
    20: (50, 100),   # Deakin Residential Car Park
    21: (135, 175),  # Bus Stop to Box Hill
    22: (70, 90),    # Entrance to the creek
    23: (165, 60),   # Building T
    24: (160, 30),   # Building BA
    25: (180, 35),   # Building HI
    26: (130, 95),   # Wheelchair Accessible toilet Pavillion Building
    27: (40, 55),    # Building B
    28: (45, 70),    # Building A
    29: (130, 40),   # Building P
    30: (105, 100)   # Building HE
}

# Create an adjacency matrix to represent the wheelchair-accessible paths
num_locations = len(nodes)
adjacency_matrix = np.full((num_locations, num_locations), float('inf'))

# Define path connections and their distances (meters)
connections = [
    (0, 1, 10),    # Main Entrance to Morgan's Walk
    (1, 2, 15),    # Morgan's Walk to Building C
    (2, 3, 15),    # Building C to Building G
    (3, 4, 45),    # Building G to Building U
    (4, 5, 55),    # Building U to Library
    (0, 5, 35),    # Main Entrance to Library
    (0, 4, 20),    # Main Entrance to Building U
    (1, 6, 35),    # Morgan's Walk to Student Central
    (6, 7, 5),     # Student Central to Wheelchair Accessible Toilet Student Central
    (7, 8, 10),    # Wheelchair Accessible Toilet Student Central to Wheelchair Accessible Sage Cafe
    (8, 9, 55),    # Wheelchair Accessible Sage Cafe to Building LC
    (9, 10, 20),   # Building LC to Wheelchair Accessible Fusion Cafe
    (10, 11, 20),  # Wheelchair Accessible Fusion Cafe to Building LA
    (11, 12, 20),  # Building LA to Building LB
    (5, 11, 70),   # Library to Building LA
    (11, 13, 30),  # Building LA to Building EC
    (13, 14, 20),  # Building EC to Building EA
    (14, 15, 32),  # Building EA to Car Park MA
    (15, 16, 27),  # Car Park MA to Building MB
    (16, 17, 14),  # Building MB to Deakin Active Gym
    (16, 18, 20),  # Building MB to Building MC
    (17, 18, 34),  # Deakin Active Gym to Building MC
    (18, 19, 27),  # Building MC to Residential Basketball Court
    (19, 20, 8),   # Residential Basketball Court to Deakin Residential Car Park
    (20, 7, 98),   # Deakin Residential Car Park to Wheelchair Accessible Toilet Student Central
    (6, 9, 40),    # Student Central to Building LC
    (9, 13, 35),   # Building LC to Building EC
    (3, 6, 50),    # Building G to Student Central
    (1, 20, 108),  # Morgan's Walk to Deakin Residential Car Park
    (2, 8, 64),    # Building C to Wheelchair Accessible Sage Cafe
    (4, 10, 90),   # Building U to Wheelchair Accessible Fusion Cafe
    (15, 18, 38),  # Car Park MA to Building MC
    (12, 14, 35),  # Building LB to Building EA
    (15, 21, 15),  # Car Park MA to Bus Stop to Box Hill
    (20, 22, 15),  # Deakin Residential Car Park to Entrance to the Creek
    (4, 26, 25),   # Building U to Wheelchair Accessible toilet Pavillion Ground
    (1, 27, 30),   # Morgan's Walk to Building B
    (1, 28, 35),   # Morgan's Walk to Building A
    (27, 28, 5),   # Building B to Building A
    (5, 23, 15),   # Library to Building T
    (5, 24, 20),   # Library to Building BA
    (5, 25, 30),   # Library to Building HI
    (23, 24, 10),  # Building T to Building BA
    (24, 25, 10),  # Building BA to Building HI
    (5, 29, 15),   # Library to Building P
    (29, 6, 15),   # Building P to Student Central
    (6, 30, 5),    # Student Central to Building HE
    (25, 6, 60),   # Building HI to Student Central
]

# Define terrain features for path segments (edges)
# Types: 'flat', 'slope', 'obstacles', 'kerb_ramp', 'crossing'
terrain_features = {
    (0, 1): {'type': 'flat', 'effort_multiplier': 1.0},
    (1, 2): {'type': 'mild_slope', 'effort_multiplier': 1.3},
    (2, 3): {'type': 'flat', 'effort_multiplier': 1.0},
    (3, 4): {'type': 'flat', 'effort_multiplier': 1.0},
    (4, 5): {'type': 'flat', 'effort_multiplier': 1.0},
    (1, 6): {'type': 'mild_slope', 'effort_multiplier': 1.3},
    (6, 7): {'type': 'flat', 'effort_multiplier': 1.0},
    (7, 8): {'type': 'flat', 'effort_multiplier': 1.0},
    (8, 9): {'type': 'gravel', 'effort_multiplier': 1.6},
    (9, 10): {'type': 'flat', 'effort_multiplier': 1.0},
    (10, 11): {'type': 'flat', 'effort_multiplier': 1.0},
    (11, 12): {'type': 'flat', 'effort_multiplier': 1.0},
    (5, 11): {'type': 'flat', 'effort_multiplier': 1.0},
    (11, 13): {'type': 'kerb_ramp', 'effort_multiplier': 1.4},
    (13, 14): {'type': 'steep_slope', 'effort_multiplier': 1.8},
    (14, 15): {'type': 'flat', 'effort_multiplier': 1.0},
    (15, 16): {'type': 'flat', 'effort_multiplier': 1.0},
    (16, 17): {'type': 'kerb_ramp', 'effort_multiplier': 1.4},
    (17, 18): {'type': 'mild_slope', 'effort_multiplier': 1.3},
    (18, 19): {'type': 'crossing', 'effort_multiplier': 1.5},
    (19, 20): {'type': 'flat', 'effort_multiplier': 1.0},
    (20, 7): {'type': 'mild_slope', 'effort_multiplier': 1.3},
    (6, 9): {'type': 'flat', 'effort_multiplier': 1.0},
    (9, 13): {'type': 'steep_slope', 'effort_multiplier': 1.8},
    (3, 6): {'type': 'mild_slope', 'effort_multiplier': 1.3},
    (1, 20): {'type': 'gravel', 'effort_multiplier': 1.6},
    (2, 8): {'type': 'steep_slope', 'effort_multiplier': 1.8},
    (4, 10): {'type': 'kerb_ramp', 'effort_multiplier': 1.4},
    (15, 18): {'type': 'gravel', 'effort_multiplier': 1.6},
    (12, 14): {'type': 'mild_slope', 'effort_multiplier': 1.3},
    (0, 19): {'type': 'mild_slope', 'effort_multiplier': 1.3},
    (5, 10): {'type': 'flat', 'effort_multiplier': 1.0},
    (2, 20): {'type': 'crossing', 'effort_multiplier': 1.5},
    (6, 14): {'type': 'steep_slope', 'effort_multiplier': 1.8},
    (8, 17): {'type': 'gravel', 'effort_multiplier': 1.6},
    (10, 13): {'type': 'kerb_ramp', 'effort_multiplier': 1.4},
    (12, 13): {'type': 'flat', 'effort_multiplier': 1.0},
    (7, 19): {'type': 'flat', 'effort_multiplier': 1.0},
    (3, 9): {'type': 'mild_slope', 'effort_multiplier': 1.3},
    (16, 19): {'type': 'steep_slope', 'effort_multiplier': 1.8},
    (4, 6): {'type': 'crossing', 'effort_multiplier': 1.5},
    (0, 2): {'type': 'mild_slope', 'effort_multiplier': 1.3},
}

# Fill the adjacency matrix with the connection distances
for start, end, distance in connections:
    adjacency_matrix[start, end] = distance
    adjacency_matrix[end, start] = distance  # Assuming bidirectional paths
    
    # Add matching terrain feature for reverse direction if not already defined
    if (end, start) not in terrain_features and (start, end) in terrain_features:
        terrain_features[(end, start)] = terrain_features[(start, end)]

# Original A* Heuristic: Euclidean distance
def original_heuristic(node, goal):
    """Calculate the Euclidean distance between two nodes"""
    x1, y1 = coordinates[node]
    x2, y2 = coordinates[goal]
    return np.sqrt((x2 - x1)**2 + (y2 - y1)**2)

# New A* Heuristic: Accounts for terrain features
def enhanced_heuristic(node, goal):
    """
    Enhanced heuristic considering terrain difficulty.
    It adds a weighted Euclidean distance by estimating effort due to terrain.
    """
    x1, y1 = coordinates[node]
    x2, y2 = coordinates[goal]
    base_distance = np.sqrt((x2 - x1)**2 + (y2 - y1)**2)
    
    # Estimate average terrain multiplier toward the goal
    multiplier = 1.0
    for neighbor in range(num_locations):
        if adjacency_matrix[node][neighbor] != float('inf'):
            if (node, neighbor) in terrain_features:
                multiplier = max(multiplier, terrain_features[(node, neighbor)]['effort_multiplier'])

    return base_distance * multiplier

# A* Algorithm Implementation with selectable heuristic
def a_star(start, goal, use_enhanced_heuristic=False, consider_terrain=False):
    """
    A* algorithm implementation for finding the optimal wheelchair-accessible path
    
    Parameters:
    - start: Starting node ID
    - goal: Goal node ID
    - use_enhanced_heuristic: Whether to use the enhanced terrain-aware heuristic
    - consider_terrain: Whether to account for terrain features in path cost
    
    Returns:
    - path: List of node IDs from start to goal
    - distance: Total physical distance of the path
    - effort: Total effort cost considering terrain (if enabled)
    """
    if start not in range(num_locations) or goal not in range(num_locations):
        return None, float('inf'), float('inf')
    
    # Choose appropriate heuristic function
    heuristic_fn = enhanced_heuristic if use_enhanced_heuristic else original_heuristic
    
    # Initialize data structures
    open_set = []
    closed_set = set()
    g_scores = {start: 0}
    f_scores = {start: heuristic_fn(start, goal)}
    came_from = {}
    effort_scores = {start: 0}
    
    # Add start node to open set
    heapq.heappush(open_set, (f_scores[start], start))
    
    while open_set:
        # Get node with lowest f score
        current_f, current = heapq.heappop(open_set)
        
        # Check if we've reached the goal
        if current == goal:
            # Reconstruct path
            path = []
            while current in came_from:
                path.append(current)
                current = came_from[current]
            path.append(start)
            path.reverse()
            
            return path, g_scores[goal], effort_scores[goal]
        
        # Add current node to closed set
        closed_set.add(current)
        
        # Check all neighbors
        for neighbor in range(num_locations):
            # Skip if no direct connection or already evaluated
            if adjacency_matrix[current, neighbor] == float('inf') or neighbor in closed_set:
                continue
            
            # Get basic distance
            distance = adjacency_matrix[current, neighbor]
            
            # Calculate terrain-adjusted effort if enabled
            if consider_terrain and (current, neighbor) in terrain_features:
                effort = distance * terrain_features[(current, neighbor)]['effort_multiplier']
            else:
                effort = distance
            
            # Calculate scores
            tentative_g = g_scores[current] + distance
            tentative_effort = effort_scores[current] + effort
            
            # Check if this path is better
            if neighbor not in g_scores or tentative_g < g_scores[neighbor]:
                # Update path
                came_from[neighbor] = current
                g_scores[neighbor] = tentative_g
                effort_scores[neighbor] = tentative_effort
                
                # Choose appropriate cost for f-score calculation
                path_cost = tentative_effort if consider_terrain else tentative_g
                f_scores[neighbor] = path_cost + heuristic_fn(neighbor, goal)
                
                # Add to open set if not already there
                if not any(node == neighbor for _, node in open_set):
                    heapq.heappush(open_set, (f_scores[neighbor], neighbor))
    
    # No path found
    return None, float('inf'), float('inf')

# Dijkstra's Algorithm Implementation (Task 4.1)
def dijkstra(start, goal, consider_terrain=False):
    """
    Dijkstra's algorithm implementation for finding the optimal wheelchair-accessible path
    
    Parameters:
    - start: Starting node ID
    - goal: Goal node ID
    - consider_terrain: Whether to account for terrain features in path cost
    
    Returns:
    - path: List of node IDs from start to goal
    - distance: Total physical distance of the path
    - effort: Total effort cost considering terrain (if enabled)
    """
    if start not in range(num_locations) or goal not in range(num_locations):
        return None, float('inf'), float('inf')
    
    # Initialize data structures
    priority_queue = []
    distances = {i: float('inf') for i in range(num_locations)}
    distances[start] = 0
    effort_scores = {i: float('inf') for i in range(num_locations)}
    effort_scores[start] = 0
    previous = {i: None for i in range(num_locations)}
    visited = set()
    
    # Add start node to priority queue
    heapq.heappush(priority_queue, (0, start))  # (distance, node)
    
    while priority_queue:
        # Get node with lowest distance
        current_distance, current = heapq.heappop(priority_queue)
        
        # Skip if already processed
        if current in visited:
            continue
        
        # Check if we've reached the goal
        if current == goal:
            # Reconstruct path
            path = []
            while current is not None:
                path.append(current)
                current = previous[current]
            path.reverse()
            
            return path, distances[goal], effort_scores[goal]
        
        # Mark as visited
        visited.add(current)
        
        # Check all neighbors
        for neighbor in range(num_locations):
            # Skip if no direct connection
            if adjacency_matrix[current, neighbor] == float('inf'):
                continue
            
            # Get basic distance
            distance = adjacency_matrix[current, neighbor]
            
            # Calculate terrain-adjusted effort if enabled
            if consider_terrain and (current, neighbor) in terrain_features:
                effort = distance * terrain_features[(current, neighbor)]['effort_multiplier']
            else:
                effort = distance
            
            # Calculate new distance
            new_distance = distances[current] + distance
            new_effort = effort_scores[current] + effort
            
            # Update if we found a better path
            if new_distance < distances[neighbor]:
                distances[neighbor] = new_distance
                effort_scores[neighbor] = new_effort
                previous[neighbor] = current
                
                # Use priority based on terrain if enabled, otherwise use distance
                priority = new_effort if consider_terrain else new_distance
                heapq.heappush(priority_queue, (priority, neighbor))
    
    # No path found
    return None, float('inf'), float('inf')


def compare_navigation():
    print("\nWheelchair-Accessible Campus Navigation")
    print("----------------------------------------")

    # Print available locations
    print("\nAvailable Locations:")
    for node_id, name in nodes.items():
        print(f"{node_id}: {name}")
    
    while True:
        try:
            # User input for start and goal
            start_input = input("\nEnter the number for START location (or type 'exit' to quit): ").strip()
            if start_input.lower() == 'exit':
                print("Thank you for using the campus navigation system")
                break
            start = int(start_input)

            goal_input = input("Enter the number for DESTINATION location: ").strip()
            goal = int(goal_input)

            if start not in nodes or goal not in nodes:
                print("Invalid input. Please enter valid location numbers.")
                continue
                
            # A* algorithm
            path1, distance1, effort1 = a_star(start, goal, use_enhanced_heuristic=True, consider_terrain=True)
            
            # Dijkstra algorithm
            path2, distance2, effort2 = dijkstra(start, goal, consider_terrain=True)

            print(f"\nPath from {nodes[start]} to {nodes[goal]}:")
            if path1 and path2:
                # Display A* results
                path1_names = [nodes[node] for node in path1]
                print("\n1. A* Algorithm:")
                print(" -> ".join(path1_names))
                print(f"Distance: {distance1} meters (Effort: {effort1:.2f})")
                
                # Display Dijkstra results
                path2_names = [nodes[node] for node in path2]
                print("\n2. Dijkstra Algorithm:")
                print(" -> ".join(path2_names))
                print(f"Distance: {distance2} meters (Effort: {effort2:.2f})")
            else:
                print("No path found between the selected locations.")

        except ValueError:
            print("Invalid input. Please enter numeric IDs.")

# Run the comparison
if __name__ == "__main__":
    compare_navigation()


Wheelchair-Accessible Campus Navigation
----------------------------------------

Available Locations:
0: Main Entrance
1: Morgan's Walk
2: Building C
3: Building G
4: Building U
5: Library
6: Student Central
7: Wheelchair Accessible Toilet Student Central
8: Wheelchair Accessible Sage Cafe
9: Building LC
10: Wheelchair Accessible Fusion Cafe
11: Building LA
12: Building LB
13: Building EC
14: Building EA
15: Car Park MA
16: Building MB
17: Deakin Active Gym
18: Building MC
19: Residential Basketball Court
20: Deakin Residential Car Park
21: Bus Stop to Box Hill
22: Entrance to the creek
23: Building T
24: Building BA
25: Building HI
26: Wheelchair Accessible toilet Pavillion Building
27: Building B
28: Building A
29: Building P
30: Building HE



Enter the number for START location (or type 'exit' to quit):  15
Enter the number for DESTINATION location:  27



Path from Car Park MA to Building B:

1. A* Algorithm:
Car Park MA -> Building MC -> Residential Basketball Court -> Deakin Residential Car Park -> Morgan's Walk -> Building B
Distance: 211.0 meters (Effort: 312.10)

2. Dijkstra Algorithm:
Car Park MA -> Building EA -> Building EC -> Building LC -> Student Central -> Morgan's Walk -> Building B
Distance: 192.0 meters (Effort: 246.50)



Enter the number for START location (or type 'exit' to quit):  exit


Thank you for using the campus navigation system
